In [2]:
pip install requests beautifulsoup4 pandas tqdm

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install selenium

In [9]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

def crawl_journalist_info(office_id):
    url = f'https://media.naver.com/journalists/whole?officeId={office_id}'
    
    driver = webdriver.Chrome()  # Change to the appropriate driver based on your browser.
    driver.get(url)
    
    # Wait for the page to load (adjust as needed)
    driver.implicitly_wait(5)
    
    journalist_list = []

    while True:
        # Get the initial page height
        last_height = driver.execute_script("return document.body.scrollHeight")

        # Scroll to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Wait for a short moment to let the page content load
        time.sleep(1)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break

        # Update last height for the next iteration
        last_height = new_height

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    for journalist_item in soup.select('.journalist_list_content_body'):
        # 1. 기자 식별번호(JournalistNumber)와 기자 이름(Journalist) 추출
        subscribe_channel_key = journalist_item.select_one('.subscribe._my_feed_btn')['data-subscribechannelkey']
        journalist_number = subscribe_channel_key.split('_')[-1]

        subscribe_name = journalist_item.select_one('.subscribe._my_feed_btn')['data-subscribename']
        journalist_name = subscribe_name

        # 2. 소속 언론사(Press) 추출
        press_tag = journalist_item.select_one('.journalist_list_content_info_item')
        press = press_tag.get_text(strip=True) if press_tag else ''

        # 3. 카테고리(Category) 추출
        category_tag = press_tag.find_next('span')
        category = category_tag.get_text(strip=True) if category_tag else ''

        # 정보를 딕셔너리로 저장
        journalist_info = {
            'JournalistNumber': journalist_number,
            'Journalist': journalist_name,
            'Press': press,
            'Category': category
        }

        journalist_list.append(journalist_info)

    driver.quit()

    return journalist_list

# officeId에 언론사 식별번호를 지정하여 크롤링 실행
office_id = '032'  # 예시로 '경향신문' 언론사의 officeId를 사용했습니다. 원하는 언론사의 officeId로 변경하세요.
journalists_info = crawl_journalist_info(office_id)

# 결과를 DataFrame으로 변환
df = pd.DataFrame(journalists_info)

# DataFrame을 CSV 파일로 저장
csv_filename = f'journalists_info_{office_id}.csv'
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f'Journalists information saved to {csv_filename}.')

Journalists information saved to journalists_info_032.csv.


In [10]:
import pandas as pd

# CSV 파일 읽기
csv_path = 'C:/Users/may_2/Documents/23-2 산정관/팀플/모으기/journalists_info_032.csv'  # 파일 경로에 맞게 수정하세요.
df = pd.read_csv(csv_path, encoding='utf-8')

# 데이터프레임 출력
df.head(20)

,JournalistNumber,Journalist,Press,Category
0,76306,이두리,경향신문,정치
1,72194,박상영,경향신문,경제
2,73769,김은성,경향신문,경제
3,74935,조해람,경향신문,사회
4,78499,김나연,경향신문,사회
5,78584,김서영,경향신문,세계
6,56121,박경은,경향신문,생활/문화
7,73386,박광연,경향신문,정치
8,79474,유새슬,경향신문,정치
9,62118,반기웅,경향신문,경제


In [15]:
pip install --upgrade selenium

Note: you may need to restart the kernel to use updated packages.


In [23]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from tqdm import tqdm
import time

def crawl_journalist_data(journalist_info, press_mapping):
    journalist_number = journalist_info['JournalistNumber']
    journalist_name = journalist_info['Journalist']
    press_name = journalist_info['Press']

    # PressNumber 찾기
    press_number = press_mapping.get(press_name, '')

    # PressNumber가 없으면 크롤링을 진행하지 않음
    if not press_number:
        print(f"Skipping {journalist_name} from {press_name}")
        return None

    # 크롤링할 사이트 주소
    url = f'https://media.naver.com/journalist/{press_number}/{journalist_number}'

    # 웹드라이버 설정 (Chrome 드라이버 사용)
    driver = webdriver.Chrome()

    try:
        driver.get(url)
        wait = WebDriverWait(driver, 10)

        # 이메일 주소 추출
        try:
            email_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.media_reporter_email')))
            email = email_element.get_attribute('href').split(':')[1]
        except Exception as e:
            email = ''
            print(f"Error extracting email for {journalist_name}: {str(e)}")

        # 기사 작성 건수 추출
        try:
            summary_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.media_reporter_summary_item em')))
            monthly_article_written = int(summary_element.text.split()[1])
        except Exception as e:
            monthly_article_written = ''
            print(f"Error extracting monthly article count for {journalist_name}: {str(e)}")

        # 결과 반환
        result = {
            'JournalistNumber': journalist_number,
            'Journalist': journalist_name,
            'PressNumber': press_number,
            'Email': email,
            'MonthlyArticleWritten': monthly_article_written
        }

        return result
    except Exception as e:
        print(f"Error crawling data for {journalist_name}: {str(e)}")
        return None
    finally:
        driver.quit()

# CSV 파일 읽기
csv_path = 'C:/Users/may_2/Documents/23-2 산정관/팀플/모으기/journalists_info_032x.csv'
df = pd.read_csv(csv_path, encoding='utf-8')

# 언론사 대상을 설정하는 매핑
press_mapping = {
    '중앙일보': '025',
    '조선일보': '023',
    '한겨레': '028',
    '동아일보': '020',
    '경향신문': '032'
}

# 크롤링 결과를 저장할 리스트
journalist_data_list = []

# tqdm을 이용한 진행 상황 로딩바 표시
for index, row in tqdm(df.iterrows(), total=len(df), desc='Crawling Journalist Data'):
    journalist_data = crawl_journalist_data(row, press_mapping)
    if journalist_data:
        journalist_data_list.append(journalist_data)

# 결과를 DataFrame으로 변환
result_df = pd.DataFrame(journalist_data_list)

# 결과를 CSV 파일로 저장
result_csv_path = 'C:/Users/may_2/Documents/23-2 산정관/팀플/모으기/journalist_result_032.csv'
result_df.to_csv(result_csv_path, index=False, encoding='utf-8-sig')

print(f'Journalist data saved to {result_csv_path}.')

Crawling Journalist Data: 100%|██████████████████████████████████████████████████████████| 5/5 [00:39<00:00,  7.86s/it]

Journalist data saved to C:/Users/may_2/Documents/23-2 산정관/팀플/모으기/journalist_result_032.csv.
